In [35]:
import subprocess
import pandas as pd
import os
import joblib
import shutil

In [36]:
java_code_folder = "C:/Users/rachi/OneDrive/Desktop/MyFolder/CodeSmell - 3/data/java"

ck_jar_path = 'ck-0.7.1.jar'
ck_output_folder = 'ck_output'
combined_dataset_file = 'combined_code_metrics.csv'
pipeline_path = 'output/code_smell_preprocessing_pipeline.joblib'
features_path = 'output/selected_features.joblib'
model_path = 'output/code_smell_detection_model.joblib'

In [37]:
print("Running CK tool to extract metrics...")
os.makedirs(ck_output_folder, exist_ok=True)

ck_command = [
    'java',
    '-jar',
    os.path.join("..", ck_jar_path),
    java_code_folder,
    'false',
    '0',
    'false'
]

try:
    subprocess.run(ck_command, check=True, capture_output=True, text=True, cwd=ck_output_folder)
    print(f"CK tool finished. Output files saved in: {ck_output_folder}")
except subprocess.CalledProcessError as e:
    print(f"Error running CK tool: {e}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
    os.path.exists(ck_output_folder) and shutil.rmtree(ck_output_folder)

Running CK tool to extract metrics...
CK tool finished. Output files saved in: ck_output


In [38]:
print("Running Code Smell Detection...", end="\n\n")

try:
    loaded_pipeline = joblib.load(pipeline_path)
    selected_features = joblib.load(features_path)
    loaded_model = joblib.load(model_path)

    new_data = pd.read_csv(os.path.join(ck_output_folder, "class.csv"))

    missing_features = [feature for feature in selected_features if feature not in new_data.columns]
    if missing_features:
        print(f"Error: The following selected features are missing in the new data: {missing_features}")
        print("Please ensure the CK output contains the necessary metrics.")

    new_data_selected = new_data[selected_features]
    new_data_processed = loaded_pipeline.transform(new_data_selected)
    predictions = loaded_model.predict(new_data_processed)

    print(f"Code Smell Predictions for the Java classes ({java_code_folder}):")
    for index, prediction in enumerate(predictions):
        class_name = new_data['class'].iloc[index]
        print(f"Class: {class_name}, Predicted Label: {prediction}")

except FileNotFoundError as e:
    print(f"Error: Could not find saved model or features: {e.filename}")
except Exception as e:
    print(f"An error occurred during prediction: {e}")
finally:
    os.path.exists(ck_output_folder) and shutil.rmtree(ck_output_folder)

Running Code Smell Detection...

Code Smell Predictions for the Java classes (C:/Users/rachi/OneDrive/Desktop/MyFolder/CodeSmell - 3/data/java):
Class: ConcreteHandler2, Predicted Label: 0
Class: ConcreteHandler1, Predicted Label: 0
Class: ChainOfResponsibilityExample, Predicted Label: 0
Class: OrderProcessor, Predicted Label: 1
Class: AbstractHandler, Predicted Label: 0
